In [1]:
import pandas as pd

Открываем то, что скачали из pcibex. Перед этим в excel удалили некоторые столбцы, чтобы оставить только те данные, которые необходимы. Ещё мы удалили всё то, что касается информации аудио и строки формата: категория - unchecked. Так как само значение у нас находится в столбце value.

In [157]:
df = pd.read_csv('results.csv', sep = ';')

In [158]:
df

,Label,PennElementName,Parameter,Value,Time,Group,Item,Expected
0,PennController,0,_Trial_,Start,1702820669477,NaN,NaN,NaN
1,Scale,inputAgree,С инструкцией ознакомился(-лась)%2C согласие н...,checked,1702820678503,Status,NaN,NaN
2,Scale,inputAgree,Choice,С инструкцией ознакомился(-лась)%2C согласие н...,1702820677536,NaN,NaN,NaN
3,PennController,0,_Trial_,End,1702820678503,NaN,NaN,NaN
4,PennController,1,_Trial_,Start,1702820678506,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11709,Selector,answer,Selection,a2,1703673010437,A,3.0,a1
11710,PennController,5,_Trial_,End,1703673010442,A,3.0,a1
11711,PennController,10,_Trial_,Start,1703673010442,A,8.0,a4
11712,Selector,answer,Selection,a4,1703673025802,A,8.0,a4


In [161]:
df.to_excel("results_filter.xlsx")

In [225]:
df = pd.read_excel('results_filter.xlsx') #в некоторых местах пропала "с инструкцией..." пришлось руками вставлять

Следующая ячейка кода направлена на то, чтобы посчитать количество правильных ответов в филлерах. Мы уберём тех людей, которые плохо справились с заданием.

In [226]:
dict_person = {}
i = 0
sum = 0
flag = 0

for index, row in df.iterrows():
    if i not in dict_person:
        dict_person[i] = []
        dict_person[i].append(index)
    
    if 'С инструкцией ознакомился(-лась)%2C согласие на обработку персональных данных даю.' in row['Value']:
        if flag == 1:
            dict_person[i].append(sum)
            dict_person[i].append(group)
            i+=1
            sum = 0
            flag = 0
    elif len(row['Value']) == 2 and (not row['Value'].isnumeric()):
        if row['Item'] > 6 and row['Item'] < 19:
            if row['Value'] == row['Expected']:
                sum+=1
                group = row['Group']
                
    elif row['Value'] != 'Start' and row['Value'] != 'End':
        dict_person[i].append(row['Value'])
        flag = 1
        
    if flag == 1 and index == 10910: #чтобы записать последнего челика
        dict_person[i].append(sum)
        dict_person[i].append(group)

In [227]:
dict_person

{0: [0, 'Женщина', '22', 'высшее%2C 15', 19, 'A'],
 1: [85, 'Женщина', '18', 'неоконченное высшее%2C 13', 16, 'A'],
 2: [168, 'Женщина', '19', 'неоконченное высшее%2C 12', 4, 'B'],
 3: [209, 'Женщина', '19', 'неоконченное высшее%2C 13', 11, 'D'],
 4: [271, 'Мужчина', '21', 'неоконченное высшее%2C 13', 18, 'A'],
 5: [354, 'Мужчина', '18', 'неоконченное высшее%2C 11', 4, 'B'],
 6: [395, 'Женщина', '19', 'высшее%2C 13', 11, 'C'],
 7: [457, 'Мужчина', '42', 'Высшее%2C 15', 12, 'D'],
 8: [519,
  'Предпочитаю не указывать',
  '18',
  'неоконченное высшее%2C 12',
  19,
  'A'],
 9: [602, 'Женщина', '18', 'неоконченное высшее%2C 11%2C5', 4, 'B'],
 10: [643, 'Мужчина', '42', 'неоконченное высшее', 9, 'C'],
 11: [705, 'Женщина', '20', 'неоконченное высшее%2C 13', 9, 'D'],
 12: [767, 'Женщина', '20', 'неоконченное высшее%2C 13', 18, 'A'],
 13: [850, 'Мужчина', '50', 'ученая степень%2C 11', 5, 'B'],
 14: [891,
  'Мужчина',
  '49',
  'Высшее%2C 15%2Cхотя вообще больше%2C можно сказать 18 +работа в о

Тут мы выделяем этих людей.

In [228]:
count = 0
dict_not_use = {}

for key, value in dict_person.items():
    if value[5] == 'A' and value[4] < 14:
        dict_not_use[key] = value
        count+=1
    if value[5] == 'B' and value[4] < 3:
        dict_not_use[key] = value
        count+=1
    if (value[5] == 'C' or value[5] == 'D') and value[4] < 9:
        dict_not_use[key] = value
        count+=1
        
print(count)

30


In [229]:
dict_not_use

{16: [1015, 'Женщина', '49', '2 высших%2C 17 лет', 10, 'A'],
 32: [1986, 'Мужчина', '50', 'высшее%2C 20', 12, 'A'],
 42: [2627, 'Женщина', '35', 'Высшее%2C 15', 7, 'C'],
 52: [3226, 'Женщина', '52', 'Среднее', 11, 'A'],
 55: [3454, 'Мужчина', '45', 'Высшее 23', 8, 'D'],
 59: [3765, 'Женщина', '72', 'Высшее%2C15', 11, 'A'],
 62: [4014, 'Мужчина', '72', '10', 13, 'A'],
 63: [4097, 'Мужчина', '48', 'Среднее-специальное%2C 11', 8, 'D'],
 64: [4159, 'Женщина', '41', 'Среднее специальное%2C 15', 12, 'A'],
 65: [4242, 'Женщина', '66', 'Среднее ', 2, 'B'],
 81: [5129, 'Женщина', '45', 'Среднее специальное%2C15', 2, 'B'],
 84: [5294, 'Женщина', '19', 'Среднее%2C3', 6, 'C'],
 85: [5356,
  'Предпочитаю не указывать',
  '64',
  'Средне- специальное%2C 13',
  4,
  'C'],
 98: [6162, 'Мужчина', '43', 'Высшее 15', 6, 'C'],
 103: [6451, 'Женщина', '63', '15', 2, 'B'],
 109: [6802, 'Женщина', '60', 'Высшее', 11, 'A'],
 110: [6885, 'Мужчина', '35', 'высшее%2C 16', 8, 'C'],
 116: [7236, 'Мужчина', '43', '

In [230]:
df.to_excel("1.xlsx") #посмотрим, сколько надо прибавлять по индексу, чтобы удалить

Удаляем с помощью индексов строк. 

Начиная от индекс из словаря - 2, заканчивая индекс + 80 для А

индекс из словаря - 2 : индекс + 38 для B

индекс из словаря - 2 : индекс + 59 для С и D

In [231]:
df_for_count = df.copy()

In [232]:
df_for_count

,Label,PennElementName,Parameter,Value,Time,Group,Item,Expected
0,PennController,0,_Trial_,Start,1.702821e+12,NaN,NaN,NaN
1,Scale,inputAgree,Choice,С инструкцией ознакомился(-лась)%2C согласие н...,1.702821e+12,NaN,NaN,NaN
2,PennController,0,_Trial_,End,1.702821e+12,NaN,NaN,NaN
3,PennController,1,_Trial_,Start,1.702821e+12,NaN,NaN,NaN
4,Scale,inputGender,Choice,Женщина,1.702821e+12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
10906,Selector,answer,Selection,a2,1.703666e+12,D,3.0,a3
10907,PennController,5,_Trial_,End,1.703666e+12,D,3.0,a3
10908,PennController,10,_Trial_,Start,1.703666e+12,D,8.0,a4
10909,Selector,answer,Selection,a4,1.703666e+12,D,8.0,a4


In [234]:
for key, value in dict_not_use.items():
    st = value[0] - 2
    if value[5] == 'A':
        fin = value[0] + 80
    if value[5] == 'B':
        fin = value[0] + 38
    if (value[5] == 'C' or value[5] == 'D'):
        fin = value[0] + 59
    st_del = list(range(st, fin+1))
    df_for_count = df_for_count.drop(index=st_del, axis = 0)

In [235]:
df_for_count.to_excel("2.xlsx") #вновь проверяем всё и ручками удаляем "предпочитаю не указывать"

In [3]:
df_for_stat = pd.read_excel('2.xlsx')

Вот мы всё удалили. Теперь перейдём к форматированию таблицы.

In [4]:
df_for_stat

,Label,PennElementName,Parameter,Value,Time,Group,Item,Expected
0,PennController,0,_Trial_,Start,1.702821e+12,NaN,NaN,NaN
1,Scale,inputAgree,Choice,С инструкцией ознакомился(-лась)%2C согласие н...,1.702821e+12,NaN,NaN,NaN
2,PennController,0,_Trial_,End,1.702821e+12,NaN,NaN,NaN
3,PennController,1,_Trial_,Start,1.702821e+12,NaN,NaN,NaN
4,Scale,inputGender,Choice,Женщина,1.702821e+12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
8798,Selector,answer,Selection,a2,1.703666e+12,D,3.0,a3
8799,PennController,5,_Trial_,End,1.703666e+12,D,3.0,a3
8800,PennController,10,_Trial_,Start,1.703666e+12,D,8.0,a4
8801,Selector,answer,Selection,a4,1.703666e+12,D,8.0,a4


С помощью этого словаря сделаем так, чтобы ответы у пользователей разделялись + фиксируем время.

In [9]:
dict_person_for_stat = {}
i = 0
sum = 0
flag = 0

for index, row in df_for_stat.iterrows():
    if i not in dict_person_for_stat:
        dict_person_for_stat[i] = []
        answers = {}
    if len(row['Value']) == 2 and (not row['Value'].isnumeric()):
        if row['Item'] < 7:
            group = row['Group']
            t = int(df_for_stat.iloc[index-1]['Time']) - int(df_for_stat.iloc[index+1]['Time']) 
            answers[row['Item']] = [row['Value'], row['Expected'], t]
    else:
        if row['PennElementName'] == 'inputEducation':
            stroka = row['Value'].split(' ')
            if len(stroka) > 1 and stroka[-1].isnumeric():
                new_str = []
                for el in stroka:
                    el_2 = el.replace('%2C', '')
                    new_str.append(el_2)
                dict_person_for_stat[i].append(new_str[-1])
                stroka = new_str[:-1]
                stroka = ' '.join(stroka)
                dict_person_for_stat[i].append(stroka)
            else:
                dict_person_for_stat[i].append('NaN')
                stroka = ' '.join(stroka)
                dict_person_for_stat[i].append(stroka)
        elif 'С инструкцией' not in row['Value'] and row['Value'] != 'Start' and row['Value'] != 'End':
            dict_person_for_stat[i].append(row['Value'])
        elif 'С инструкцией' in row['Value'] and index != 1:
            dict_person_for_stat[i].append(group)
            dict_person_for_stat[i].append(answers)
            i+=1
    if index == 8802: #чтобы записать последнего челика
        dict_person_for_stat[i].append(group)
        dict_person_for_stat[i].append(answers)

In [10]:
dict_person_for_stat

{0: ['Женщина',
  '22',
  '15',
  'высшее',
  'A',
  {6.0: ['a2', 'a2', -19685],
   1.0: ['a2', 'a3', -9026],
   3.0: ['a1', 'a1', -10743],
   4.0: ['a3', 'a1', -14220],
   5.0: ['a2', 'a2', -13412],
   2.0: ['a2', 'a3', -8495]}],
 1: ['Женщина',
  '18',
  '13',
  'неоконченное высшее',
  'A',
  {3.0: ['a1', 'a1', -9913],
   5.0: ['a2', 'a2', -8019],
   2.0: ['a2', 'a3', -7939],
   1.0: ['a3', 'a3', -10906],
   4.0: ['a1', 'a1', -12476],
   6.0: ['a2', 'a2', -9431]}],
 2: ['Женщина',
  '19',
  '12',
  'неоконченное высшее',
  'B',
  {2.0: ['a3', 'a3', -13235],
   3.0: ['a1', 'a1', -12300],
   4.0: ['a1', 'a1', -13606],
   5.0: ['a2', 'a2', -7116],
   1.0: ['a2', 'a3', -10832],
   6.0: ['a2', 'a2', -10463]}],
 3: ['Женщина',
  '19',
  '13',
  'неоконченное высшее',
  'D',
  {6.0: ['a2', 'a2', -10339],
   2.0: ['a1', 'a1', -9297],
   5.0: ['a2', 'a1', -7869],
   4.0: ['a2', 'a2', -9565],
   1.0: ['a1', 'a3', -13351],
   3.0: ['a2', 'a3', -10315]}],
 4: ['Мужчина',
  '21',
  '13',
  'неок

In [12]:
Gender = []
Age = []
age_ed = []
education = []
Group = []
answers = []

for key, value in dict_person_for_stat.items():
    if len(value) > 2:
        Gender.append(value[0])
        Age.append(value[1])
        age_ed.append(value[2])
        education.append(value[3])
        Group.append(value[4])
        answers.append(value[5])

In [13]:
df_for_stat = pd.DataFrame(list(zip(Gender, Age, age_ed, education, Group, answers)),
               columns =['Gender', 'Age', 'age_ed', 'education', 'Group', 'answers'])

In [14]:
df_for_stat

,Gender,Age,age_ed,education,Group,answers
0,Женщина,22,15,высшее,A,"{6.0: ['a2', 'a2', -19685], 1.0: ['a2', 'a3', ..."
1,Женщина,18,13,неоконченное высшее,A,"{3.0: ['a1', 'a1', -9913], 5.0: ['a2', 'a2', -..."
2,Женщина,19,12,неоконченное высшее,B,"{2.0: ['a3', 'a3', -13235], 3.0: ['a1', 'a1', ..."
3,Женщина,19,13,неоконченное высшее,D,"{6.0: ['a2', 'a2', -10339], 2.0: ['a1', 'a1', ..."
4,Мужчина,21,13,неоконченное высшее,A,"{1.0: ['a3', 'a3', -23260], 5.0: ['a2', 'a2', ..."
...,...,...,...,...,...,...
138,Мужчина,29,NaN,Высшее,C,"{5.0: ['a2', 'a1', -14612], 6.0: ['a1', 'a2', ..."
139,Женщина,37,15,Среднее специальное,D,"{3.0: ['a4', 'a3', -17402], 1.0: ['a1', 'a3', ..."
140,Мужчина,36,15,Среднее-специальное,A,"{1.0: ['a2', 'a3', -13060], 3.0: ['a1', 'a1', ..."
141,Мужчина,21,NaN,15,C,"{3.0: ['a2', 'a3', -11621], 1.0: ['a2', 'a3', ..."


In [15]:
df_for_stat.to_excel("output_for_stat.xlsx") 

In [16]:
df_for_stat = pd.read_excel('output_for_stat_clean.xlsx') #ручками почистить образование

In [17]:
df_for_stat

,Gender,Age,age_ed,education,Group,answers
0,Женщина,22,15.0,высшее,A,"{6.0: ['a2', 'a2', 19685.0], 1.0: ['a2', 'a3',..."
1,Женщина,18,13.0,неоконченное высшее,A,"{3.0: ['a1', 'a1', 9913.0], 5.0: ['a2', 'a2', ..."
2,Женщина,19,12.0,неоконченное высшее,B,"{2.0: ['a3', 'a3', 13235.0], 3.0: ['a1', 'a1',..."
3,Женщина,19,13.0,неоконченное высшее,D,"{6.0: ['a2', 'a2', 10339.0], 2.0: ['a1', 'a1',..."
4,Мужчина,21,13.0,неоконченное высшее,A,"{1.0: ['a3', 'a3', 23260.0], 5.0: ['a2', 'a2',..."
...,...,...,...,...,...,...
138,Мужчина,29,NaN,Высшее,C,"{5.0: ['a2', 'a1', 14612.0], 6.0: ['a1', 'a2',..."
139,Женщина,37,15.0,Среднее специальное,D,"{3.0: ['a4', 'a3', 17402.0], 1.0: ['a1', 'a3',..."
140,Мужчина,36,15.0,Среднее-специальное,A,"{1.0: ['a2', 'a3', 13060.0], 3.0: ['a1', 'a1',..."
141,Мужчина,21,15.0,высшее,C,"{3.0: ['a2', 'a3', 11621.0], 1.0: ['a2', 'a3',..."


In [18]:
df_for_stat['Age'] = df_for_stat['Age'].astype(int)

Всё по возрасту

In [19]:
df_for_stat['Age'].mean()

36.40559440559441

In [20]:
df_for_stat['Age'].median()

38.0

In [21]:
df_for_stat['Age'].std()

15.232118476133069

In [22]:
df_for_stat['Age'].mode()

0    20
Name: Age, dtype: int32

In [23]:
##ничего не работает, хотя я так всегда считаю chi... может, он и не нужен здесь?
# exp = np.full((1, 140), 50, dtype=int)

# from scipy.stats import chisquare
# chisquare(f_obs=age, f_exp=exp)

По гендеру только количество мужчин и женщин

In [24]:
df_for_stat['Gender'].value_counts()

Женщина    83
Мужчина    60
Name: Gender, dtype: int64

Теперь работать с ответами

In [34]:
q1 = []
q2 = []
q3 = []
q4 = []
q5 = []
q6 = []
q1_p = []
q2_p = []
q3_p = []
q4_p = []
q5_p = []
q6_p = []
t1 = []
t2 = []
t3 = []
t4 = []
t5 = []
t6 = []

for index, row in df_for_stat.iterrows():
    r = eval(row['answers'])
    
    for element in r:
        
        answ = r[element]
        
        if int(float(element)) == 1:
            q1.append(answ[0])
            t1.append(answ[2])
            if answ[0] == answ[1]:
                q1_p.append(1)
            else:
                q1_p.append(0)
        elif int(float(element)) == 2:
            q2.append(answ[0])
            t2.append(answ[2])
            if answ[0] == answ[1]:
                q2_p.append(1)
            else:
                q2_p.append(0)
        elif int(float(element)) == 3:
            q3.append(answ[0])
            t3.append(answ[2])
            if answ[0] == answ[1]:
                q3_p.append(1)
            else:
                q3_p.append(0)
        elif int(float(element)) == 4:
            q4.append(answ[0])
            t4.append(answ[2])
            if answ[0] == answ[1]:
                q4_p.append(1)
            else:
                q4_p.append(0)
        elif int(float(element)) == 5:
            q5.append(answ[0])
            t5.append(answ[2])
            if answ[0] == answ[1]:
                q5_p.append(1)
            else:
                q5_p.append(0)
        else:
            q6.append(answ[0])
            t6.append(answ[2])
            if answ[0] == answ[1]:
                q6_p.append(1)
            else:
                q6_p.append(0)

In [35]:
df_for_stat['q_1'] = q1
df_for_stat['q_2'] = q2
df_for_stat['q_3'] = q3
df_for_stat['q_4'] = q4
df_for_stat['q_5'] = q5
df_for_stat['q_6'] = q6
df_for_stat['q_1_p'] = q1_p
df_for_stat['q_2_p'] = q2_p
df_for_stat['q_3_p'] = q3_p
df_for_stat['q_4_p'] = q4_p
df_for_stat['q_5_p'] = q5_p
df_for_stat['q_6_p'] = q6_p
df_for_stat['t_1'] = t1
df_for_stat['t_2'] = t2
df_for_stat['t_3'] = t3
df_for_stat['t_4'] = t4
df_for_stat['t_5'] = t5
df_for_stat['t_6'] = t6

In [36]:
df_for_stat

,Gender,Age,age_ed,education,Group,answers,q_1,q_2,q_3,q_4,...,q_3_p,q_4_p,q_5_p,q_6_p,t_1,t_2,t_3,t_4,t_5,t_6
0,Женщина,22,15.0,высшее,A,"{6.0: ['a2', 'a2', 19685.0], 1.0: ['a2', 'a3',...",a2,a2,a1,a3,...,1,0,1,1,9026.0,8495.0,10743.0,14220.0,13412.0,19685.0
1,Женщина,18,13.0,неоконченное высшее,A,"{3.0: ['a1', 'a1', 9913.0], 5.0: ['a2', 'a2', ...",a3,a2,a1,a1,...,1,1,1,1,10906.0,7939.0,9913.0,12476.0,8019.0,9431.0
2,Женщина,19,12.0,неоконченное высшее,B,"{2.0: ['a3', 'a3', 13235.0], 3.0: ['a1', 'a1',...",a2,a3,a1,a1,...,1,1,1,1,10832.0,13235.0,12300.0,13606.0,7116.0,10463.0
3,Женщина,19,13.0,неоконченное высшее,D,"{6.0: ['a2', 'a2', 10339.0], 2.0: ['a1', 'a1',...",a1,a1,a2,a2,...,0,1,0,1,13351.0,9297.0,10315.0,9565.0,7869.0,10339.0
4,Мужчина,21,13.0,неоконченное высшее,A,"{1.0: ['a3', 'a3', 23260.0], 5.0: ['a2', 'a2',...",a3,a1,a1,a1,...,1,1,1,1,23260.0,20021.0,10044.0,10417.0,14685.0,8624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Мужчина,29,NaN,Высшее,C,"{5.0: ['a2', 'a1', 14612.0], 6.0: ['a1', 'a2',...",a2,a2,a2,a2,...,0,1,0,0,19586.0,17454.0,12962.0,12942.0,14612.0,10900.0
139,Женщина,37,15.0,Среднее специальное,D,"{3.0: ['a4', 'a3', 17402.0], 1.0: ['a1', 'a3',...",a1,a1,a4,a1,...,0,0,1,1,13298.0,13871.0,17402.0,14750.0,9698.0,9883.0
140,Мужчина,36,15.0,Среднее-специальное,A,"{1.0: ['a2', 'a3', 13060.0], 3.0: ['a1', 'a1',...",a2,a1,a1,a2,...,1,0,1,0,13060.0,9061.0,18019.0,13842.0,10143.0,8136.0
141,Мужчина,21,15.0,высшее,C,"{3.0: ['a2', 'a3', 11621.0], 1.0: ['a2', 'a3',...",a2,a1,a2,a2,...,0,1,0,1,11773.0,12697.0,11621.0,9137.0,8921.0,9000.0


Штуки по правильным ответам. Если значения высокие, значит пользователи шли за контекстом и ориентировались именно на него

In [48]:
df_for_stat.mean()

C:\Users\пользователь\AppData\Local\Temp\ipykernel_2488\3893505765.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_for_stat.mean()


Age       36.864286
age_ed    15.283333
q_1_p      0.214286
q_2_p      0.314286
q_3_p      0.492857
q_4_p      0.521429
q_5_p      0.542857
q_6_p      0.685714
dtype: float64

In [49]:
df_for_stat.std()

C:\Users\пользователь\AppData\Local\Temp\ipykernel_2488\375359819.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_for_stat.std()


Age       15.267029
age_ed     3.462120
q_1_p      0.411799
q_2_p      0.465898
q_3_p      0.501744
q_4_p      0.501334
q_5_p      0.499949
q_6_p      0.465898
dtype: float64

In [80]:
df_for_stat.groupby('Group')[['q_1_p', 'q_2_p', 'q_3_p', 'q_4_p', 'q_5_p', 'q_6_p']].mean()

,q_1_p,q_2_p,q_3_p,q_4_p,q_5_p,q_6_p
Group,,,,,,
A,0.242424,0.121212,0.727273,0.484848,0.666667,0.636364
B,0.190476,0.261905,0.785714,0.428571,0.690476,0.619048
C,0.206897,0.586207,0.241379,0.586207,0.482759,0.724138
D,0.222222,0.333333,0.138889,0.611111,0.305556,0.777778


In [88]:
df_for_stat.groupby('Group')[['q_1_p', 'q_2_p', 'q_3_p', 'q_4_p', 'q_5_p', 'q_6_p']].std()

,q_1_p,q_2_p,q_3_p,q_4_p,q_5_p,q_6_p
Group,,,,,,
A,0.435194,0.331434,0.452267,0.507519,0.478714,0.488504
B,0.397437,0.445001,0.415300,0.500870,0.467901,0.491507
C,0.412251,0.501230,0.435494,0.501230,0.508548,0.454859
D,0.421637,0.478091,0.350736,0.494413,0.467177,0.421637


In [68]:
df_for_stat.groupby('Group')[['q_1']].value_counts()

Group  q_1 
A      'a2'    16
       'a1'     8
       'a3'     8
       'a4'     1
B      'a2'    23
       'a1'    10
       'a3'     8
       'a4'     1
C      'a2'    18
       'a3'     6
       'a1'     4
       'a4'     1
D      'a2'    21
       'a3'     8
       'a1'     5
       'a4'     2
dtype: int64

In [69]:
df_for_stat.groupby('Group')[['q_2']].value_counts()

Group  q_2 
A      'a2'    15
       'a1'    13
       'a3'     4
       'a4'     1
B      'a1'    14
       'a2'    14
       'a3'    11
       'a4'     3
C      'a1'    17
       'a2'     7
       'a3'     5
D      'a2'    18
       'a1'    12
       'a3'     6
dtype: int64

In [70]:
df_for_stat.groupby('Group')[['q_3']].value_counts()

Group  q_3 
A      'a1'    24
       'a2'     6
       'a3'     2
       'a4'     1
B      'a1'    33
       'a2'     4
       'a3'     4
       'a4'     1
C      'a1'    14
       'a2'     8
       'a3'     7
D      'a2'    25
       'a1'     5
       'a3'     5
       'a4'     1
dtype: int64

In [71]:
df_for_stat.groupby('Group')[['q_4']].value_counts()

Group  q_4 
A      'a1'    16
       'a4'     7
       'a2'     6
       'a3'     4
B      'a1'    18
       'a2'    12
       'a3'     7
       'a4'     5
C      'a2'    17
       'a1'     6
       'a3'     4
       'a4'     2
D      'a2'    22
       'a1'     7
       'a3'     5
       'a4'     2
dtype: int64

In [72]:
df_for_stat.groupby('Group')[['q_5']].value_counts()

Group  q_5 
A      'a2'    22
       'a3'     5
       'a1'     4
       'a4'     2
B      'a2'    29
       'a1'     5
       'a3'     4
       'a4'     4
C      'a1'    14
       'a2'    10
       'a3'     5
D      'a2'    18
       'a1'    11
       'a3'     5
       'a4'     2
dtype: int64

In [73]:
df_for_stat.groupby('Group')[['q_6']].value_counts()

Group  q_6 
A      'a2'    21
       'a3'     5
       'a1'     4
       'a4'     3
B      'a2'    26
       'a1'     7
       'a3'     7
       'a4'     2
C      'a2'    21
       'a1'     4
       'a3'     2
       'a4'     2
D      'a2'    28
       'a1'     6
       'a3'     2
dtype: int64

In [77]:
df_for_stat.groupby('Gender')[['q_1_p', 'q_2_p', 'q_3_p', 'q_4_p', 'q_5_p', 'q_6_p']].mean()

,q_1_p,q_2_p,q_3_p,q_4_p,q_5_p,q_6_p
Gender,,,,,,
Женщина,0.139535,0.302326,0.523256,0.523256,0.511628,0.651163
Мужчина,0.333333,0.333333,0.444444,0.518519,0.592593,0.740741


In [78]:
df_for_stat.groupby('Gender')[['q_1_p', 'q_2_p', 'q_3_p', 'q_4_p', 'q_5_p', 'q_6_p']].std()

,q_1_p,q_2_p,q_3_p,q_4_p,q_5_p,q_6_p
Gender,,,,,,
Женщина,0.348536,0.461959,0.502388,0.502388,0.502797,0.479398
Мужчина,0.475831,0.475831,0.501570,0.504349,0.495966,0.442343


In [79]:
df_for_stat['Group'].value_counts()

B    42
D    36
A    33
C    29
Name: Group, dtype: int64

А вот возраст/пол/образование хочу на графиках посмотреть. Мне кажется, что это будет эффективнее, чем пытаться что-то высчитать.Только у меня всегда плохо с выбором графиков...

In [94]:
import matplotlib.pyplot as plt

Только перед этим приведу всё образование к строчным буквам

In [92]:
df_for_stat['education'] = df_for_stat['education'].str.lower()

Милена, у тебя была визуализация в прошлом году, может быть, ты построишь графики? (если это не нагло с моей стороны)

In [100]:
df_for_stat.to_excel("output_with_ans.xlsx") 

In [23]:
dict_fail_vv_qq = {} #если чел допустил ошибку при соответствии
dict_cont_interpret_vq_qv = {} #если чел пошёл за контекстом
dict_aud_interpret_vq_qv = {} #если чел пошёл за аудио
dict_bag_aud = {} #если чел пошёл за аудио или вообще ничего не понял
dict_bag_right = {} #если чел на контекст обратил внимание

for index, row in df_for_stat.iterrows():
    if row['Group'] == 'A':
        i = 0
        for element in row['questions']:
            if element == 4 or element == 6:
                if row['righ_or_not'][i] == 0:
                    if index not in dict_fail_vv_qq:
                        dict_fail_vv_qq[index] = []
                        dict_fail_vv_qq[index].append(element)
                    else:
                        dict_fail_vv_qq[index].append(element)
            
            elif element == 3 or element == 5:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_cont_interpret_vq_qv:
                        dict_cont_interpret_vq_qv[index] = []
                        dict_cont_interpret_vq_qv[index].append(element)
                    else:
                        dict_cont_interpret_vq_qv[index].append(element)
                else:
                    if index not in dict_aud_interpret_vq_qv:
                        dict_aud_interpret_vq_qv[index] = []
                        dict_aud_interpret_vq_qv[index].append(element)
                    else:
                        dict_aud_interpret_vq_qv[index].append(element)
            
            else:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_bag_right:
                        dict_bag_right[index] = []
                        dict_bag_right[index].append(element)
                    else:
                        dict_bag_right[index].append(element)
                else:
                    if index not in dict_bag_aud:
                        dict_bag_aud[index] = []
                        dict_bag_aud[index].append(element)
                    else:
                        dict_bag_aud[index].append(element)
                    
            i+=1
    if row['Group'] == 'B':
        i = 0
        for element in row['questions']:
            if element == 3 or element == 5:
                if row['righ_or_not'][i] == 0:
                    if index not in dict_fail_vv_qq:
                        dict_fail_vv_qq[index] = []
                        dict_fail_vv_qq[index].append(element)
                    else:
                        dict_fail_vv_qq[index].append(element)
            
            elif element == 4 or element == 6:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_cont_interpret_vq_qv:
                        dict_cont_interpret_vq_qv[index] = []
                        dict_cont_interpret_vq_qv[index].append(element)
                    else:
                        dict_cont_interpret_vq_qv[index].append(element)
                else:
                    if index not in dict_aud_interpret_vq_qv:
                        dict_aud_interpret_vq_qv[index] = []
                        dict_aud_interpret_vq_qv[index].append(element)
                    else:
                        dict_aud_interpret_vq_qv[index].append(element)
            
            else:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_bag_right:
                        dict_bag_right[index] = []
                        dict_bag_right[index].append(element)
                    else:
                        dict_bag_right[index].append(element)
                else:
                    if index not in dict_bag_aud:
                        dict_bag_aud[index] = []
                        dict_bag_aud[index].append(element)
                    else:
                        dict_bag_aud[index].append(element)
            i+=1
    if row['Group'] == 'C':
        i = 0
        for element in row['questions']:
            if element == 4 or element == 5:
                if row['righ_or_not'][i] == 0:
                    if index not in dict_fail_vv_qq:
                        dict_fail_vv_qq[index] = []
                        dict_fail_vv_qq[index].append(element)
                    else:
                        dict_fail_vv_qq[index].append(element)
            
            elif element == 2 or element == 6:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_cont_interpret_vq_qv:
                        dict_cont_interpret_vq_qv[index] = []
                        dict_cont_interpret_vq_qv[index].append(element)
                    else:
                        dict_cont_interpret_vq_qv[index].append(element)
                else:
                    if index not in dict_aud_interpret_vq_qv:
                        dict_aud_interpret_vq_qv[index] = []
                        dict_aud_interpret_vq_qv[index].append(element)
                    else:
                        dict_aud_interpret_vq_qv[index].append(element)
            
            else:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_bag_right:
                        dict_bag_right[index] = []
                        dict_bag_right[index].append(element)
                    else:
                        dict_bag_right[index].append(element)
                else:
                    if index not in dict_bag_aud:
                        dict_bag_aud[index] = []
                        dict_bag_aud[index].append(element)
                    else:
                        dict_bag_aud[index].append(element)
            i+=1
    if row['Group'] == 'D':
        i = 0
        for element in row['questions']:
            if element == 2 or element == 6:
                if row['righ_or_not'][i] == 0:
                    if index not in dict_fail_vv_qq:
                        dict_fail_vv_qq[index] = []
                        dict_fail_vv_qq[index].append(element)
                    else:
                        dict_fail_vv_qq[index].append(element)
            
            elif element == 4 or element == 5:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_cont_interpret_vq_qv:
                        dict_cont_interpret_vq_qv[index] = []
                        dict_cont_interpret_vq_qv[index].append(element)
                    else:
                        dict_cont_interpret_vq_qv[index].append(element)
                else:
                    if index not in dict_aud_interpret_vq_qv:
                        dict_aud_interpret_vq_qv[index] = []
                        dict_aud_interpret_vq_qv[index].append(element)
                    else:
                        dict_aud_interpret_vq_qv[index].append(element)
            
            else:
                if row['righ_or_not'][i] != 0:
                    if index not in dict_bag_right:
                        dict_bag_right[index] = []
                        dict_bag_right[index].append(element)
                    else:
                        dict_bag_right[index].append(element)
                else:
                    if index not in dict_bag_aud:
                        dict_bag_aud[index] = []
                        dict_bag_aud[index].append(element)
                    else:
                        dict_bag_aud[index].append(element)
            i+=1